In [15]:
from __future__ import annotations
from typing import Any
from pydantic import BaseModel
from pydantic._internal._model_construction import ModelMetaclass
from typing import Dict

class SignalDescriptor:
    """Return `$Model.field` on the class, real value on an instance."""

    def __init__(self, field_name: str) -> None:
        self.field_name = field_name

    def __get__(self, instance, owner):
        #  class access  →  owner is the model class, instance is None
        if instance is None:
            config = getattr(owner, "model_config", {})
            print(config)
            ns = config.get("namespace", owner.__name__)
            use_ns = config.get("use_namespace", False)
            return f"${ns}.{self.field_name}" if use_ns else f"${self.field_name}"
        return instance.__dict__[self.field_name]        

        #  instance access  →  behave like a normal attribute

class SignalModelMeta(ModelMetaclass):
    def __init__(cls, name, bases, ns, **kw):
        super().__init__(name, bases, ns, **kw)

        # For each declared field, replace the stub Pydantic left in the
        # class __dict__ with our custom descriptor
        for field_name in cls.model_fields:
            setattr(cls, field_name, SignalDescriptor(field_name))
            setattr(cls, f"{field_name}_signal", SignalDescriptor(field_name))

class User(BaseModel, metaclass=SignalModelMeta):
    id: int
    name: str
    age: int

    model_config = {
        "namespace": "User",
        "use_namespace": True,
    }

    @property
    def signals(self) -> Dict[str, Any]:
        return self.model_dump()
    
    @property
    def signals_ns(self) -> Dict[str, Any]:
        "namespaced signals"
        return {self.__class__.__name__:self.signals}
    
    @property
    def scope(self) -> str:
        "scope as string"
        return self.__class__._get_config_value("scope", "Unnamed")
    
    @classmethod
    def _get_config_value(cls, key: str, default=None):
        """Get configuration value from model_config."""
        return cls.model_config.get(f"faststate_{key}", default)
    
    def signal(self, field: str) -> Any:
        if field in self.signals.keys():
            return f"${self.__class__.__name__}.{field}"
        else:
            raise ValueError(f"Field {field} not found in {self.__class__.__name__}")

user = User(id=1, name="John", age=30)
user.signals_ns
user.signals
user.signal("id")

user.model_dump()

User.id_signal

{'namespace': 'User', 'use_namespace': True}


'$User.id'

In [18]:
user.signals_ns

{'User': {'id': 1, 'name': 'John', 'age': 30}}